In [1]:
import os, pickle

In [2]:
with open("../../../training_data/8.Apos/Extra_set/features.pkl", "rb") as f:
    extras_featuresd = pickle.load(f)

len(extras_featuresd), extras_featuresd

(9,
 {'8sgj':     Residues                                                          \
           pdb label_entity_id label_asym_id label_seq_id auth_asym_id   
  0       8sgj               1             A           52            A   
  1       8sgj               1             A           53            A   
  2       8sgj               1             A           54            A   
  3       8sgj               1             A           55            A   
  4       8sgj               1             A           56            A   
  ..       ...             ...           ...          ...          ...   
  746     8sgj               1             A          941            A   
  747     8sgj               1             A          942            A   
  748     8sgj               1             A          943            A   
  749     8sgj               1             A          944            A   
  750     8sgj               1             A          945            A   
  
                       

# Make predictions

deepallo was run in its own environment in /data/fnerin/envs_other_tools, created with the requirements_conda.txt file

In [3]:
os.system(f"ln -s {os.getcwd().rsplit('/', 3)[0]}/training_data/utils/external/fpocket/bin/fpocket fpocket") # os.getcwd().rsplit('/', 3)[0] --> absolute path of allodb_new

ln: failed to create symbolic link 'fpocket': File exists


256

In [4]:
for pdb, feats in extras_featuresd.items():
    if not os.path.isfile(f"{pdb}/{pdb}.pkl"):
        os.makedirs(pdb, exist_ok=True)
    
        # The following is skipped because the script is hardcoded for rcsb-downloaded .pdb files
        # if not os.path.isfile(f"{pdb}/{pdb}.pdb"):
        #     os.system(f"ln -s {os.getcwd().rsplit('/', 1)[0]}/structures/{pdb}.pdb {pdb}/{pdb}.pdb")
        #     # os.getcwd().rsplit('/', 1)[0] --> absolute path of previous directory
    
        os.system(f"python deepallo/inference.py {pdb} {feats[('Residues', 'auth_asym_id')].unique().item()}") # If it is not a single chain, .item() will fail

Fetching 328 files:   0%|          | 0/328 [00:00<?, ?it/s]/data/fnerin/envs_other_tools/deepallo/lib/python3.11/site-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
/data/fnerin/envs_other_tools/deepallo/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 328 files: 100%|██████████| 328/328 [00:00<00:00, 3250.05it/s]
Traceback (most recent call last):
  File "/home/fnerin/Desktop/

## AlphaFold model

- Needs the SEQRES in a .pdb; AT LEAST
- Edited utils.sequence_indices to effectively get the .cif from EBI.

In [5]:
from Bio.Data import IUPACData

In [7]:
pdb = "AF-A0A1D8PQM9-F1"
feats = extras_featuresd[pdb]

seq = """
MSTKTDISTEELVSLFSKAGFEEKKSAEIVKNKKVASALYNILGSNFPKTDDKKLSLLHQLAIHESKNGKVPNHDFIIDG
IQKGDLKTALQVTEGIKYLQNNATVDKEKFDEASGVGIEITPEEAKVEISKYLDSIKTDLESKRYSILPKVLGEVKTQPS
LKWAPPNLFKPILDEEFLARLGPKDERDVKKKEKKAKTPANAAATKKQDTGSEPERSMFSEGFLGDLHKPGEEPQMYPEL
LEEHKKFICDKVYTRFPPEPNGFLHIGHSKAIMVNFGYAQFNKGNCYLRFDDTNPEAEEEVYFNSIKEMVSWLGYKPWKI
TYSSDYFDELYELAIKLIKSDKAYICHCTPEEVKASRGLKEDGTLGGERVACKHRFQTVEHNLREFENMKNGKYNVGEAT
LRMKQDLNSPSPQMWDLVAYRVLNTPHHRTGDKWKIYPTYDFTHCLVDSFENITHSLCTTEFVLSRESYEWLCDALHVYR
PAQREYGRLNLTGTIMSKRKIAKLVNEGYVRGWDDPRLYTLEGIKRRGVPPGAILSFINTLGVTTSTTNIQTVRFESAVR
NYLDQTTPRLMMVLHPIEVVIDNLDESFSLDVEIPYKPGKDEKSMGYRKLTFSKHIYIDENDVRAEPADKEFYRLAPGQP
VGLMRVPFNISFKSIEEKDGKKIVHVNYDEGVKAKPKTYIQWIPKDTAVHIKEVRIYNQLFKSENPSAHPEGYLKDINPD
SEEVLRNAVVEENLKDIVAKSPMNIEIPGSAFNIKENKGNNTVRFQALREGYFCLDKDSKEDGLILNRIVSLKEDAAKK
""".strip().replace("\n", "")

os.makedirs(pdb, exist_ok=True)


with (
    open("../structures/AF-A0A1D8PQM9-F1.pdb", "r") as orig_pdbf,
    open(f"{pdb}/{pdb}.pdb", "w") as pdbf
):
    for i in range(0, len(seq), 13):
        chunk = seq[i : i + 13]
        resnames = " ".join(IUPACData.protein_letters_1to3[a].upper() for a in chunk)
        pdbf.write(f"SEQRES  {i//13+1:>2} A  {len(seq):>3}  {resnames}\n")

    pdbf.write(orig_pdbf.read())

In [9]:
os.system(f"python deepallo/inference.py {pdb} {feats[('Residues', 'auth_asym_id')].unique().item()}") # If it is not a single chain, .item() will fail

/data/fnerin/envs_other_tools/deepallo/lib/python3.11/site-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
/data/fnerin/envs_other_tools/deepallo/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 328 files: 100%|██████████| 328/328 [00:00<00:00, 3474.14it/s]
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no

(0.06609036028385162, ['794', '718', '527', '698', '697', '517', '516', '791', '793', '790', '702', '557', '519', '552', '699', '700', '560', '788', '493', '701', '789', '799', '556', '553'])
(0.04113222658634186, ['495', '497', '707', '491', '494', '496', '500', '490'])
(0.03209031745791435, ['705', '704', '491', '707', '549', '494', '492', '490'])


0

## 8vw5

Does not have a legacy .pdb to download.

In [6]:
pdb = "8vw5"
feats = extras_featuresd[pdb]

seq = """
GSPKQAAADRRTVEKTWKLMDKVVRLCQNPKLQLKNSPPYILDILPDTYQHLRLILSRYDDNQKLAQLSENEYFKIYIDS
LMKKSKRAIRLFKEGKERMYEENSQDRRNLTKLSLIFSHMLAEIKAIFPNGQFQGDNFRITKADAAEFWRKFFGDKTIVP
WKVFRQCLHEVHQISSGLEAMALKSTIDLTCNDYISVFEFDIFTRLFQPWGSILRNWNFLAVTHPGYMAFLTYDEVKARL
QKYSTKPGSYIFRLSCTRLGQWAIGYVTGDGNILQTIPHNKPLFQALIDGSREGFYLYPDGRSYNPDLTG
""".strip().replace("\n", "")

os.makedirs(pdb, exist_ok=True)


with (
    open("../structures/8vw5.pdb", "r") as orig_pdbf,
    open(f"{pdb}/{pdb}.pdb", "w") as pdbf
):
    for i in range(0, len(seq), 13):
        chunk = seq[i : i + 13]
        resnames = " ".join(IUPACData.protein_letters_1to3[a].upper() for a in chunk)
        pdbf.write(f"SEQRES  {i//13+1:>2} A  {len(seq):>3}  {resnames}\n")

    pdbf.write(orig_pdbf.read())

In [7]:
os.system(f"python deepallo/inference.py {pdb} {feats[('Residues', 'auth_asym_id')].unique().item()}") # If it is not a single chain, .item() will fail

/data/fnerin/envs_other_tools/deepallo/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/data/fnerin/envs_other_tools/deepallo/lib/python3.11/site-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 328 files: 100%|██████████| 328/328 [00:00<00:00, 2861.51it/s]


***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 


mv: '8vw5/8vw5_out' and '8vw5/8vw5_out' are the same file
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Found 1 mismatches between original and current metadata:
	INFO: AutoGluon Python micro version mismatch (original=3.11.9, current=3.11.11)
/data/fnerin/envs_other_tools/deepallo/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator ExtraTreeClassifier from version 1.4.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/data/fnerin/envs_other_tools/deepallo/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator ExtraTreesClassifier from version 1.4.0 when using version 1.5.2. This might lead to breaking cod

(0.07956564426422119, ['235', '255', '218', '262', '240', '219', '232', '236', '148', '239', '263', '287', '222', '260', '149', '224', '254', '250', '215', '220', '152', '293', '145'])
(0.04762057960033417, ['291', '267', '296', '286', '268', '264', '266', '265', '148', '263', '72', '310', '290', '70', '71', '294', '295', '312', '289', '288', '144', '69'])
(0.03613067790865898, ['175', '312', '311', '314', '241', '294', '239', '238', '295', '293', '316', '313', '315'])


0

In [8]:
# Any missing
for pdb, feats in extras_featuresd.items():
    if not os.path.isfile(f"{pdb}/{pdb}.pkl"):
        print("Missing:", pdb)

# Processing

In [9]:
import pandas as pd

In [10]:
results = {
    pdb: {
        pocket: {
            "prob": pocketd["prob"],
            "residues": pd.DataFrame(pocketd["res"])
        }
        for pocket, pocketd in pd.read_pickle(f"{pdb}/{pdb}.pkl").items()
    }
    for pdb in extras_featuresd
}

len(results), results

(9,
 {'8sgj': {0: {'prob': 0.06474058330059052,
    'residues':    auth_asym_id auth_seq_id
    0             A         172
    1             A         827
    2             A          98
    3             A         837
    4             A         210
    5             A         830
    6             A         213
    7             A         103
    8             A         165
    9             A         825
    10            A         102
    11            A         224
    12            A         168
    13            A         244
    14            A         826
    15            A          99
    16            A         840
    17            A         222
    18            A         171
    19            A         833
    20            A         214
    21            A         209
    22            A         836
    23            A          97
    24            A         175
    25            A         829},
   1: {'prob': 0.0645398423075676,
    'residues':    auth_asym_id auth_se

In [11]:
from Bio import PDB

def process_pocket(f):
    s = PDB.PDBParser(QUIET=True).get_structure("protein", f)
    return pd.DataFrame((
        {
            "auth_asym_id": chain.id,
            "auth_seq_id": res.get_id()[1],
            "pdbx_PDB_ins_code": res.get_id()[2] if res.get_id()[2] != " " else "?"
        }
        for model in s
        for chain in model
        for res in chain
    ), dtype=str).drop_duplicates()

In [12]:
results = {}

for pdb in extras_featuresd:
    d = {
        pocket: {
            "pred": 1,
            "prob": pocketd["prob"],
            "residues": pd.DataFrame(pocketd["res"], dtype=str)
        }
        for pocket, pocketd in pd.read_pickle(f"{pdb}/{pdb}.pkl").items()
    }

    pdbpath = f"{pdb}/{pdb}_out/pockets"
    i = max(pocket for pocket in d.keys()) + 1
    existing = 0
    for pocketf in os.listdir(pdbpath):
        if pocketf.endswith("_atm.pdb"):
            res = process_pocket(f"{pdbpath}/{pocketf}")
            if not any(
                len( pocket["residues"].merge(res[["auth_asym_id", "auth_seq_id"]]) ) == len( pocket["residues"] )
                for pocket in d.values()
            ):
                d[i] = {
                    "pred": 0,
                    "residues": res
                }
                i += 1
            else:
                existing += 1
                
    assert existing == 3, "Not all + pred pockets were picked up"
    results[pdb] = d

len(results), results

(9,
 {'8sgj': {0: {'pred': 1,
    'prob': 0.06474058330059052,
    'residues':    auth_asym_id auth_seq_id
    0             A         172
    1             A         827
    2             A          98
    3             A         837
    4             A         210
    5             A         830
    6             A         213
    7             A         103
    8             A         165
    9             A         825
    10            A         102
    11            A         224
    12            A         168
    13            A         244
    14            A         826
    15            A          99
    16            A         840
    17            A         222
    18            A         171
    19            A         833
    20            A         214
    21            A         209
    22            A         836
    23            A          97
    24            A         175
    25            A         829},
   1: {'pred': 1,
    'prob': 0.0645398423075676,
    'resi

In [13]:
resultsf = "deepallo_results.pkl"

with open(resultsf, "wb") as f:
    pickle.dump(results, f)